<a href="https://colab.research.google.com/github/AvaneSauhard/HumanActivityRecognition/blob/main/HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing libraries
from google.colab import drive
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.utils import to_categorical
from sklearn.utils import resample
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Input 

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
def load_file(filepath):
  data=pd.read_csv(filepath,header=None,delim_whitespace=True)
  return data.values

In [5]:
def load_group(filenames,filepaths):
  load=list()
  for name in filenames:
    data=load_file(filepaths+name)
    load.append(data)
  load=np.dstack(load)
  return load


In [6]:
filepath_train="drive/MyDrive/UCI HAR Dataset/train/Inertial Signals/"
filename_train=list()
filename_train+=['total_acc_x_train.txt','total_acc_y_train.txt','total_acc_z_train.txt']
filename_train+=['body_acc_x_train.txt','body_acc_y_train.txt','body_acc_z_train.txt']
filename_train+=['body_gyro_x_train.txt','body_gyro_y_train.txt','body_gyro_z_train.txt']
train_X=load_group(filename_train,filepath_train)
train_Y=load_file("drive/MyDrive/UCI HAR Dataset/train/y_train.txt")

In [7]:
print(train_X.shape,train_Y.shape)

(7352, 128, 9) (7352, 1)


In [8]:
filepath_test="drive/MyDrive/UCI HAR Dataset/test/Inertial Signals/"
filename_test=list()
filename_test+=['total_acc_x_test.txt','total_acc_y_test.txt','total_acc_z_test.txt']
filename_test+=['body_acc_x_test.txt','body_acc_y_test.txt','body_acc_z_test.txt']
filename_test+=['body_gyro_x_test.txt','body_gyro_y_test.txt','body_gyro_z_test.txt']
test_X=load_group(filename_test,filepath_test)
test_Y=load_file("drive/MyDrive/UCI HAR Dataset/test/y_test.txt")

In [9]:
print(test_X.shape,test_Y.shape)

(2947, 128, 9) (2947, 1)


In [10]:
train_Y=train_Y-1
test_Y=test_Y-1

In [11]:
train_Y=to_categorical(train_Y)
test_Y=to_categorical(test_Y)

In [12]:
print(train_X.shape,train_Y.shape,test_X.shape,test_Y.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [33]:
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.layers import Dense
from keras.models import Model
def build_model():
  
  timestamps, features, output=train_X.shape[1],train_X.shape[2],train_Y.shape[1]
  #head1
  inputs1= Input(shape=(timestamps,features))
  conv1 = Conv1D(filters=64,kernel_size=3,activation='relu')(inputs1)
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  #head2
  inputs2= Input(shape=(timestamps,features))
  conv2 = Conv1D(filters=64,kernel_size=5,activation='relu')(inputs2)
  drop2 = Dropout(0.5)(conv2)
  pool2 = MaxPooling1D(pool_size=2)(drop2)
  flat2 = Flatten()(pool2)
  #head3
  inputs3= Input(shape=(timestamps,features))
  conv3 = Conv1D(filters=64,kernel_size=11,activation='relu')(inputs3)
  drop3 = Dropout(0.5)(conv3)
  pool3 = MaxPooling1D(pool_size=2)(drop3)
  flat3 = Flatten()(pool3)
  merged = concatenate([flat1,flat2,flat3])
  dense = Dense(100,activation='relu')(merged)
  output = Dense(train_Y.shape[1],activation='softmax')(dense)

  model = Model(inputs=[inputs1,inputs2,inputs3],outputs=output)
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.summary()
  model.fit([train_X,train_X,train_X],train_Y,epochs=8,batch_size=64)
  return model
  

In [34]:
model = build_model()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_38 (InputLayer)          [(None, 128, 9)]     0           []                               
                                                                                                  
 input_39 (InputLayer)          [(None, 128, 9)]     0           []                               
                                                                                                  
 input_40 (InputLayer)          [(None, 128, 9)]     0           []                               
                                                                                                  
 conv1d_36 (Conv1D)             (None, 126, 64)      1792        ['input_38[0][0]']               
                                                                                           

In [35]:
model.evaluate([test_X,test_X,test_X],test_Y, batch_size = 64, verbose =0)

[0.2860216498374939, 0.9243298172950745]